# Criação do banco de dados no PostgreSQL

O banco de dados utilizado está disponível em: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce<br>
Para utilização deste script, os arquivos devem ser salvos na subpasta database.

In [43]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import pandas as pd
import os

In [104]:
# Definição de função para conectar ao banco de dados.
# A conexão está dentro de uma transaction, por isso é necessário encerrar a transaction aberta com commit.
def connect_to_database(sgbd, user, password, host, port, database):
    try:
        engine = create_engine(f"{sgbd}://{user}:{password}@{host}:{port}/{database}")
        conn = engine.connect()
        conn.execute("COMMIT")
        return conn, engine
    except Exception as error:
        print(error)

In [105]:
# Para criar um novo database, é necessário primeiramente conectar a um database existente para depois criar o novo.
# O database default postgres foi escolhido para conexão inicial.
conn, engine = connect_to_database('postgresql', 'postgres', 'postgres', 'localhost', '5433', 'postgres')

In [106]:
# Criação do novo database e encerramento da conexão com o database postgres.
try:
    conn.execute("CREATE DATABASE olist")
    conn.execute("COMMIT")
except Exception as error:
    print(error)

(psycopg2.errors.DuplicateDatabase) ERRO:  o banco de dados "olist" já existe

[SQL: CREATE DATABASE olist]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [107]:
conn.close()

In [108]:
# Estabelecimento de conexão com o novo database olist.
conn, engine = connect_to_database('postgresql', 'postgres', 'postgres', 'localhost', '5433', 'olist')

In [113]:
# Função para leitura de arquivo .csv em dataframe pandas e gravação em tabela SQL.
def csv_to_sql(csv_file, sql_table, engine, conn):
    df = pd.read_csv(csv_file)
    conn.execute(f"DROP TABLE IF EXISTS {sql_table}")
    try:
        number_of_rows = df.to_sql(name=sql_table, con=engine, index=False)
        print(f'Tabela {sql_table} criada com {number_of_rows} linhas.')
    except Exception as error:
        print(error)

In [114]:
# Recupera o nomes dos arquivos .csv na pasta indicada.
# Neste caso, todos os arquivos .csv estão gravados na subpasta database
database_files = os.listdir(os.getcwd() + "\\database")
database_files

['olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_orders_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_products_dataset.csv',
 'olist_sellers_dataset.csv']

In [116]:
# Executa a função para os arquivos .csv recuperados
# Os nomes dos arquivos .csv seguem o padrão 'olist_<nome da tabela>_database.csv'
# Assim, determinamos os nomes das tabelas na variável sql_table a partir dos nomes dos arquivos.
for csv_file in database_files:
    sql_table = csv_file[csv_file.index('_')+1:len(csv_file)-csv_file[::-1].index('_')-1]
    csv_file = 'database/'+csv_file
    csv_to_sql(csv_file, sql_table, engine, conn)

Tabela customers criada com 441 linhas.
Tabela geolocation criada com 163 linhas.
Tabela orders criada com 441 linhas.
Tabela order_items criada com 650 linhas.
Tabela order_payments criada com 886 linhas.
Tabela order_reviews criada com 224 linhas.
Tabela products criada com 951 linhas.
Tabela sellers criada com 95 linhas.


In [117]:
# Encerra a conexão com o banco
conn.close()